# Loading Census Data From Census Reporter

Load data directly from the Census Reporter API, but use the columns titles for names, rather than the column id numbers. 

Downloading data from the Census Reporter API returns both data and metadata. 

To select an area to get results for, you must provide both a geoid for the containing area, and a summary level for the subdivisions. 

To get the summary level, see this [spreadsheet of summary levels.](https://docs.google.com/spreadsheets/d/1gdaIim_V-zcBqQa8aaKaft-DMzdewb5FDF1-jztZvw4/edit?usp=sharing). You can get the geoids by browsing and searching at the [Census Reporter website](http://censusreporter.org)


In [60]:
import requests #  run '!pip install requests' in the Notebook to install
import pandas as pd

# Get the Age by sex table, and a lot of metadata, directly from the Census Reporter API
# for all tracts in California

table_id = 'B01001'
summary_level = '140' # Tracts
geoid='16000US0666000' # San Diego

data = requests.get("http://api.censusreporter.org/1.0/data/show/latest"
                    "?table_ids={table_id}&geo_ids={sl}|{geoid}"
                    .format(table_id=table_id, sl=summary_level, geoid=geoid)).json()

In [63]:
from operator import itemgetter

# It looks like the JSON dicts may be properly sorted, but I'm not sure I can rely on that.
# So, sort the column id values, then make a columns title list in the same order
columns = data['tables']['B01001']['columns']
columns_id_names = sorted(columns.keys() )
column_titles = [ columns[e]['name'] for e in columns_id_names]

# The item getter will produce a tuple from a dict, getting all of the 
# dict items in the order of the keys named in the argument list. In this case, 
# it turns the dict into a tuple of estimate values
row_ig = itemgetter(*columns_id_names)

tracts = data['data'].keys()

# Actually get the row data. 
estimate_rows = [ row_ig(data['data'][tract]['B01001']['estimate']) for tract in tracts ]
error_rows = [ row_ig(data['data'][tract]['B01001']['error']) for tract in tracts ]
# Now, creating the dataframe is really easy. 
df = pd.DataFrame(estimate_rows, columns=column_titles)
df.head()

,Total:,Male:,Under 5 years,5 to 9 years,10 to 14 years,15 to 17 years,18 and 19 years,20 years,21 years,22 to 24 years,...,50 to 54 years,55 to 59 years,60 and 61 years,62 to 64 years,65 and 66 years,67 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
0,4530.0,2225.0,103.0,185.0,140.0,47.0,48.0,51.0,58.0,129.0,...,131.0,73.0,74.0,19.0,7.0,17.0,22.0,6.0,35.0,21.0
1,5429.0,2627.0,273.0,189.0,171.0,89.0,109.0,61.0,198.0,207.0,...,243.0,169.0,28.0,19.0,34.0,115.0,82.0,28.0,75.0,18.0
2,3580.0,1750.0,194.0,70.0,95.0,59.0,28.0,8.0,0.0,62.0,...,180.0,151.0,82.0,17.0,23.0,55.0,77.0,72.0,111.0,79.0
3,4099.0,2133.0,165.0,71.0,37.0,72.0,17.0,13.0,0.0,118.0,...,102.0,59.0,19.0,50.0,60.0,76.0,33.0,0.0,17.0,61.0
4,6064.0,3146.0,138.0,97.0,80.0,45.0,0.0,8.0,38.0,321.0,...,85.0,132.0,79.0,48.0,42.0,50.0,41.0,42.0,26.0,17.0
